In [62]:
import glob
from datetime import datetime
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats, signal
from scipy.stats import mode
from scipy.fft import fft
from sklearn.ensemble import RandomForestClassifier as RFC, ExtraTreesClassifier as ETC
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

import lightgbm as lgb
import xgboost as xgb

from tqdm import tqdm, trange

# Generic functions (usable for all test cases)

## Core functions

In [63]:
def segmentation(df, overlap_rate, time_window):
    seg_data = []
    #convert overlap rate to step for sliding window
    overlap = int((1 - overlap_rate)*time_window)
    # interpolate
    df = df.interpolate().ffill().fillna(0)
    for i in range(0, len(df)-time_window+1, overlap):
        seg_data.append(df.loc[i:i+time_window-1, :].copy().reset_index(drop=True))
    return seg_data


def rename_columns(df):
    df.columns = [
        "FH_X", "FH_Y", "FH_Z",     #1
        "TH_X", "TH_Y", "TH_Z",     #2
        "RH_X", "RH_Y", "RH_Z",     #3
        "RS_X", "RS_Y", "RS_Z",     #4
        "RO_X", "RO_Y", "RO_Z",     #5
        "RE_X", "RE_Y", "RE_Z",     #6
        "RW_X", "RW_Y", "RW_Z",     #7
        "LS_X", "LS_Y", "LS_Z",     #8
        "LE_X", "LE_Y", "LE_Z",     #9
        "LW_X", "LW_Y", "LW_Z",     #10
        "RA_X", "RA_Y", "RA_Z",     #11
        "LA_X", "LA_Y", "LA_Z",     #12
        "VS_X", "VS_Y", "VS_Z",     #13
        "subject_id", "activity",   # Other columns
    ]
    return df


def dataloader(overlap, window_size, verbose=True):
    if verbose:
        print("loading the data...", end="\t")
    data_list = []
    file_lengths = {1: [], 2: [], 3: []}
    files = tqdm(glob.glob("E:/Temp/Bento/Data/TrainData/*/*/*.csv")) if verbose else glob.glob("E:/Temp/Bento/Data/TrainData/*/*/*.csv")
    for file in files:
        tempdf = pd.read_csv(file)
        tempdf = rename_columns(tempdf)
        segmented_data = segmentation(tempdf, overlap, window_size)
        if len(segmented_data)>0:
            person = segmented_data[0].reset_index(drop=True).loc[0, "subject_id"]
            file_lengths[person].append(len(segmented_data))   
        data_list.extend(segmented_data)
    return data_list, file_lengths


def feature_extractor(data_list, verbose=True):
    if verbose:
        print(f"extracting the features...", end="  ")
    X, y = {1:[], 2:[], 3:[]}, {1:[], 2:[], 3:[]}
    num_range = trange(0,len(data_list)) if verbose else range(0,len(data_list))
    for j in num_range:
        #extract only xyz columns
        person = data_list[j].loc[0, "subject_id"]
        x_data = data_list[j].drop(columns=["subject_id","activity"])
        X[person].append(get_features(x_data))
        y[person].append(data_list[j].reset_index(drop=True).loc[0, "activity"])
    return X, y


def majority_voting(predictions, file_lengths):
    filtered_predictions = []
    index = 0
    for length in file_lengths:
        file_pred = predictions[index:index+length]
        modes = mode(file_pred)
        majority_choice = modes.mode[0]
        filtered_predictions.extend([majority_choice]*length)
        index += length
    return filtered_predictions

In [64]:
def get_processed_dataset(overlap_rate, window_size, verbose=True):
    data_list, file_lengths = dataloader(overlap_rate, window_size, verbose=verbose)
    stream_list = []
    for df in data_list:
        stream_list.append(get_streams(df))
    X, y = feature_extractor(stream_list, verbose=verbose)
    return X, y, file_lengths


def model_evaluator(model, X, y, file_lengths, n_repeats=7, voting=True, verbose=True):
    scores = []
    num_range = trange(n_repeats) if verbose else range(n_repeats)
    for _ in num_range:
        for p1, p2, p3 in [(1,2,3), (2,3,1), (3,1,2)]:
            X_test, y_test = X[p1], y[p1]
            X_train = X[p2] + X[p3]
            y_train = y[p2] + y[p3]
            # print(f"training model for person {p1}/3...", end="\t")
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            if voting:
                filtered_pred = majority_voting(pred, file_lengths[p1])
                scores.append(accuracy_score(y_test, filtered_pred))
            else:
                scores.append(accuracy_score(y_test, pred))
    if verbose:
        print(f"Mean Score: {np.mean(scores)}")
        print(f"Std Score: {np.std(scores)}")
        print(f"Min Score: {np.min(scores)}")
        print(f"Max Score: {np.max(scores)}")
    return scores


In [65]:
def LOOCV_train_evaluate(model, overlap_rate, window_size, voting=True, n_repeats=1, verbose=True):
    scores = []
    data_list, file_lengths = dataloader(overlap_rate, window_size, verbose=verbose)
    stream_list = []
    for df in data_list:
        stream_list.append(get_streams(df))
    X, y = feature_extractor(stream_list, verbose=verbose)
    num_range = trange(n_repeats) if verbose else range(n_repeats)
    for _ in num_range:
        for p1, p2, p3 in [(1,2,3), (2,3,1), (3,1,2)]:
            X_test, y_test = X[p1], y[p1]
            X_train = X[p2] + X[p3]
            y_train = y[p2] + y[p3]
            # print(f"training model for person {p1}/3...", end="\t")
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            if voting:
                filtered_pred = majority_voting(pred, file_lengths[p1])
                scores.append(accuracy_score(y_test, filtered_pred))
            else:
                scores.append(accuracy_score(y_test, pred))
    if verbose:
        print(f"\nMean Score: {np.mean(scores)}")
    return scores


def GridSearch(parameters, csvpath = "..", filename="gridCV_results", n_repeats=7, verbose=False, progress=True):
    score_df = pd.DataFrame({"model": [], "window_size": [], "overlap_rate": [], "n_repeats":[], 
                            "avg_score": [], "std_score": [], "scores":[]})
    models, window_sizes, overlap_rates = parameters["model"], parameters["window_size"], parameters["overlap_rate"]
    combinations = [(i,j,k) for i in models for j in window_sizes for k in overlap_rates]
    if progress:
        combinations = tqdm(combinations)
    for combination in combinations:
        model, window_size, overlap_rate = combination
        scores = LOOCV_train_evaluate(model, overlap_rate, window_size, n_repeats=n_repeats, verbose=verbose)
        score_df = score_df.append({"model": model.__str__(), "window_size": window_size, 
                    "overlap_rate": overlap_rate, "n_repeats": n_repeats, "avg_score": np.mean(scores),
                    "std_score": np.array(scores).std(),"scores": scores}, ignore_index=True)
    savepath = f"{csvpath}/{filename}_{str(datetime.now())[:-7]}.csv".replace(":", ".")
    score_df.to_csv(savepath, index=False)
    print(f"result exported to: {savepath}")


## Stream Extractors

In [66]:
def get_speed_acc(x_data):
    x_data = x_data.drop(columns=["activity", "subject_id"])
    speed = x_data.diff().fillna(0)
    acc = speed.diff().fillna(0)
    speed.columns = [f"{col}_speed" for col in speed.columns]
    acc.columns = [f"{col}_acc" for col in acc.columns]
    return speed, acc

def get_speed_acc_jerk(x_data):
    x_data = x_data.drop(columns=["activity", "subject_id"])
    speed = x_data.diff().fillna(0)
    acc = speed.diff().fillna(0)
    jerk = acc.diff().fillna(0)
    speed.columns = [f"{col}_speed" for col in speed.columns]
    acc.columns = [f"{col}_acc" for col in acc.columns]
    jerk.columns = [f"{col}_jerk" for col in acc.columns]
    return speed, acc, jerk

In [67]:
def joint_distance(x_data, joint1, joint2):
    """
    returns the distance between two joints. 
    """
    x1, y1, z1 = x_data[f"{joint1}_X"], x_data[f"{joint1}_Y"], x_data[f"{joint1}_Z"]
    x2, y2, z2 = x_data[f"{joint2}_X"], x_data[f"{joint2}_Y"], x_data[f"{joint2}_Z"]
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
    return distance

def get_all_joint_distances(x_data):
    """
    calculates all the necessary joint distances from the `x_data`, 
    adds columns to it and returns the modified `x_data`.
    the two joints should not be essentially consecutive, 
    because the distance between two consecutive joints is always constant.
    For example, distance between left_wrist and left_elbow is always constant.
    """
    # joints
    # Front head        ->  left shoulder       (1->8)
    x_data["dist_FH_LS"] = joint_distance(x_data, "FH", "LS")
    # Front head        ->  right shoulder      (1->4)
    x_data["dist_FH_RS"] = joint_distance(x_data, "FH", "RS")
    # left shoulder     ->  left wrist          (8->10)
    x_data["dist_LS_LW"] = joint_distance(x_data, "LS", "LW")
    # right shoulder    ->  right wrist         (4->7)
    x_data["dist_RS_RW"] = joint_distance(x_data, "RS", "RW")
    # v sacral          ->  left elbow          (13->9)
    x_data["dist_VS_LE"] = joint_distance(x_data, "VS", "LE")
    # v sacral          ->  right elbow         (13->6)
    x_data["dist_VS_RE"] = joint_distance(x_data, "VS", "RE")
    # v sacral          ->  left wrist          (13->10)
    x_data["dist_VS_LW"] = joint_distance(x_data, "VS", "LW")
    # v sacral          ->  right wrist         (13->7)
    x_data["dist_VS_RW"] = joint_distance(x_data, "VS", "RW")
    # v sacral          ->  rear head           (13->3)
    x_data["dist_VS_RH"] = joint_distance(x_data, "VS", "RH")
    # v sacral          ->  top head            (13->2)
    x_data["dist_VS_TH"] = joint_distance(x_data, "VS", "TH")
    # left wrist        ->  right wrist         (10->7)
    x_data["dist_LW_RW"] = joint_distance(x_data, "LW", "RW")
    # left asis         ->  left wrist          (12->10)
    x_data["dist_LA_LW"] = joint_distance(x_data, "LA", "LW")
    # right asis        ->  right wrist         (11->7)
    x_data["dist_RA_RW"] = joint_distance(x_data, "RA", "RW")
    # left wrist        ->  top head            (10->2)
    x_data["dist_LW_TH"] = joint_distance(x_data, "LW", "TH")
    # right wrist       ->  top head            (7->2)
    x_data["dist_RW_TH"] = joint_distance(x_data, "RW", "TH")
    # top head          ->  left asis           (2->12)
    x_data["dist_TH_LA"] = joint_distance(x_data, "TH", "LA")
    return x_data



In [68]:
def joint_angle(x_data, joint1, joint2, joint3):
    x1, y1, z1 = x_data[f"{joint1}_X"], x_data[f"{joint1}_Y"], x_data[f"{joint1}_Z"]
    x2, y2, z2 = x_data[f"{joint2}_X"], x_data[f"{joint2}_Y"], x_data[f"{joint2}_Z"]
    x3, y3, z3 = x_data[f"{joint3}_X"], x_data[f"{joint3}_Y"], x_data[f"{joint3}_Z"]
    v1 = np.array([x2-x1, y2-y1, z2-z1]).T
    v2 = np.array([x3-x2, y3-y2, z3-z2]).T
    v1_unit = v1/np.expand_dims(np.linalg.norm(v1, axis=1), axis=1)
    v2_unit = v2/np.expand_dims(np.linalg.norm(v2, axis=1), axis=1)
    angle = np.arccos(np.sum(v1_unit*v2_unit, axis=1)) # dot multiplication
    return angle

def get_all_joint_angles(x_data):
    # joints
    # left shoulder     ->  left elbow      ->  left wrist      (8->9->10)
    x_data["angle_LS_LE_LW"] = joint_angle(x_data, "LS", "LE", "LW")
    # right shoulder    ->  right elbow     ->  right wrist     (4->6->7)
    x_data["angle_RS_RE_RW"] = joint_angle(x_data, "RS", "RE", "RW")
    # right shoulder    ->  left shoulder   ->  front head      (4->8->1)
    x_data["angle_RS_LS_FH"] = joint_angle(x_data, "RS", "LS", "FH")
    # right shoulder    ->  left shoulder   ->  left elbow      (4->8->9)
    x_data["angle_RS_LS_LE"] = joint_angle(x_data, "RS", "LS", "LE")
    # left shoulder     ->  right shoulder  ->  right elbow     (8->4->6)
    x_data["angle_LS_RS_RE"] = joint_angle(x_data, "LS", "RS", "RE")
    # v sacral          ->  right offset    ->  rear head       (13->5->3)
    x_data["angle_VS_RO_RH"] = joint_angle(x_data, "VS", "RO", "RH")
    # vsacral           ->  top head        ->  front head      (13->2->1)
    x_data["angle_VS_TH_FH"] = joint_angle(x_data, "VS", "TH", "FH")
    # v sacral          ->  left shoulder   ->  left elbow      (13->8->9)
    x_data["angle_VS_LS_LE"] = joint_angle(x_data, "VS", "LS", "LE")
    # v sacral          ->  right shoulder  ->  right elbow     (13->4->6)
    x_data["angle_VS_RS_RE"] = joint_angle(x_data, "VS", "RS", "RE")
    # left asis         ->  left shoulder   ->  left elbow      (12->8->9)
    x_data["angle_LA_LS_LE"] = joint_angle(x_data, "LA", "LS", "LE")
    # right asis        -> right shoulder   ->  right elbow     (11->4->6)
    x_data["angle_RA_RS_RE"] = joint_angle(x_data, "RA", "RS", "RE")
    return x_data

In [69]:
def plane_angles(x_data, joint1, joint2):
    x1, y1, z1 = x_data[f"{joint1}_X"], x_data[f"{joint1}_Y"], x_data[f"{joint1}_Z"]
    x2, y2, z2 = x_data[f"{joint2}_X"], x_data[f"{joint2}_Y"], x_data[f"{joint2}_Z"]
    v = np.array([x2-x1, y2-y1, z2-z1]).T
    vx = np.array([1, 0, 0])
    vy = np.array([0, 1, 0])
    vz = np.array([0, 0, 1])
    v_unit = v/np.expand_dims(np.linalg.norm(v, axis=1), axis=1)
    angle_x = np.arccos(np.sum(v_unit*vx, axis=1)) 
    angle_y = np.arccos(np.sum(v_unit*vy, axis=1)) 
    angle_z = np.arccos(np.sum(v_unit*vz, axis=1)) 
    return angle_x, angle_y, angle_z

def get_all_angles_with_plane(x_data):
    # joints
    # arm angles: left shoulder     ->  left elbow 
    x_data["angle_x_LS_LE"], x_data["angle_y_LS_LE"], x_data["angle_z_LS_LE"] = plane_angles(x_data, "LS", "LE")
    # forearm angles: left elbow    ->  left  wrist
    x_data["angle_x_LE_LW"], x_data["angle_y_LE_LW"], x_data["angle_z_LE_LW"] = plane_angles(x_data, "LE", "LW")
    # arm angles: right shoulder    ->  right elbow 
    x_data["angle_x_RS_RE"], x_data["angle_y_RS_RE"], x_data["angle_z_RS_RE"] = plane_angles(x_data, "RS", "RE")
    # forearm angles: right elbow   ->  right  wrist
    x_data["angle_x_RE_RW"], x_data["angle_y_RE_RW"], x_data["angle_z_RE_RW"] = plane_angles(x_data, "RE", "RW")
    # backbone angles: v sacral     ->  rear head
    x_data["angle_x_VS_RH"], x_data["angle_y_VS_RH"], x_data["angle_z_VS_RH"] = plane_angles(x_data, "VS", "RH")
    return x_data

## Pre-processed functions

In [70]:
def current_best():
    def get_streams(x_data):
        speed, acc = get_speed_acc(x_data)
        x_data = pd.concat([x_data, speed], axis=1)
        x_data = get_all_joint_distances(x_data)
        return x_data

    def get_features(x_data):
        features = []
        cols = x_data.columns.tolist()
        #Calculate features (STD, Average, Max, Min, Median, Variance) for each data columns X Y Z 
        for k in cols:
            features.append(x_data[k].std(ddof=0))
            # features.append(np.average(x_data[k]))
            features.append(np.max(x_data[k]))
            features.append(np.min(x_data[k]))
            features.append(np.median(x_data[k]))        
            # features.append(np.var(x_data[k]))
            # fd = np.abs(fft(np.array(x_data[k])))**2
            # features.append(stats.skew(fd))
            # features.append(stats.kurtosis(fd))
        return features
    joint_names = ['FH', 'TH', 'RH', 'RS', 'RO', 'RE', 'RW', 'LS', 'LE', 'LW', 'RA', 'LA', 'VS']
    pos_cols = [i+j for i in joint_names for j in ["_X", "_Y", "_Z"]]
    speed_cols = [f"{col}_speed" for col in pos_cols]
    acc_cols = [f"{col}_acc" for col in pos_cols]
    distance_cols = ['dist_FH_LS', 'dist_FH_RS', 'dist_LS_LW', 'dist_RS_RW', 'dist_VS_LE', 'dist_VS_RE', 
        'dist_VS_LW', 'dist_VS_RW','dist_VS_RH', 'dist_VS_TH', 'dist_LW_RW', 'dist_LA_LW', 'dist_RA_RW', 
        'dist_LW_TH', 'dist_RW_TH', 'dist_TH_LA']
    colnames = pos_cols + speed_cols + acc_cols + distance_cols
    return get_streams, get_features, colnames

# Random Forest

In [71]:
get_streams, get_features, colnames = current_best()
overlap_rate, window_size = 0.8, 4000
X, y, file_lengths = get_processed_dataset(overlap_rate, window_size)

loading the data...	

100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [00:06<00:00, 21.67it/s]


extracting the features...  

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [00:22<00:00, 22.76it/s]


In [72]:
def scaled_model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True):
    scores = []
    num_range = trange(n_repeats) if verbose else range(n_repeats)
    for _ in num_range:
        for p1, p2, p3 in [(1,2,3), (2,3,1), (3,1,2)]:
            X_test, y_test = X[p1], y[p1]
            X_train = X[p2] + X[p3]
            y_train = y[p2] + y[p3]
            scaler = MinMaxScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.fit_transform(X_test)
            # print(f"training model for person {p1}/3...", end="\t")
            model.fit(X_train_scaled, y_train)
            pred = model.predict(X_test_scaled)
            if voting:
                filtered_pred = majority_voting(pred, file_lengths[p1])
                scores.append(accuracy_score(y_test, filtered_pred))
            else:
                scores.append(accuracy_score(y_test, pred))
    if verbose:
        print(f"Mean Score: {np.mean(scores)}")
        print(f"Std Score: {np.std(scores)}")
        print(f"Min Score: {np.min(scores)}")
        print(f"Max Score: {np.max(scores)}")
    return scores

In [73]:
## This cell was used for experimentation
## So far best combo: overlap_rate, window_size, n_estimators = 0.8, 4000, 2500; excluding acc

model = RFC(2500, criterion="gini", n_jobs=-1) 
scores = scaled_model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True)


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:33<00:00,  9.39s/it]

Mean Score: 0.7633523843735084
Std Score: 0.019890269355005204
Min Score: 0.7293233082706767
Max Score: 0.8053097345132744


In [43]:
model = RFC(1200, criterion="gini", max_depth=12, n_jobs=-1)
scores = model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True)


100%|██████████| 10/10 [02:05<00:00, 12.56s/it]

Mean Score: 0.715566886015833
Std Score: 0.03187665295686838
Min Score: 0.6343612334801763
Max Score: 0.7961165048543689


In [45]:
model = RFC(1200, criterion="gini", min_samples_split=6, n_jobs=-1)
scores = model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True)


100%|██████████| 10/10 [02:21<00:00, 14.17s/it]

Mean Score: 0.7183364356708543
Std Score: 0.030369686801219008
Min Score: 0.6563876651982379
Max Score: 0.7572815533980582


# Extra Trees

In [13]:
get_streams, get_features, colnames = current_best()
overlap_rate, window_size = 0.75, 3000
X, y, file_lengths = get_processed_dataset(overlap_rate, window_size)
ft_names = [i+j for i in colnames for j in ["_std", "_max", "_min", "_med"]]

loading the data...	

100%|██████████| 151/151 [00:16<00:00,  9.00it/s]


extracting the features...  

100%|██████████| 736/736 [01:18<00:00,  9.43it/s]


In [60]:
model = ETC(1800, criterion="gini", min_samples_split=4, max_depth=12, n_jobs=-1)
scores = model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True)

100%|██████████| 10/10 [02:20<00:00, 14.03s/it]

Mean Score: 0.7363534084472131
Std Score: 0.034126056701583855
Min Score: 0.6651982378854625
Max Score: 0.8203883495145631


In [72]:
model = ETC(1800, criterion="gini", max_depth=12, n_jobs=-1)
scores = model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True)


100%|██████████| 10/10 [02:11<00:00, 13.16s/it]

Mean Score: 0.7338456671893613
Std Score: 0.03239559131793469
Min Score: 0.6519823788546255
Max Score: 0.8155339805825242


In [74]:
imps = dict(zip(ft_names, model.feature_importances_))

In [84]:
with open("../imps_asc.txt", "w") as f:
    for i in pd.Series(imps).sort_values().index.tolist():
        f.write(f"{i:15}\t{imps[i]:15}\n")

In [99]:
x = X.copy()
for p in [1,2,3]:
    x[p] = pd.DataFrame(np.array(x[p]), columns=ft_names)
    x[p].drop(columns=to_be_removed, inplace=True)
    x[p] = x[p].to_numpy().tolist()
to_be_removed = pd.Series(imps).sort_values().index.tolist()[:75]


In [100]:
model = ETC(1800, criterion="gini", max_depth=12, n_jobs=-1)
scores = model_evaluator(model, x, y, file_lengths, n_repeats=10, voting=True, verbose=True)


100%|██████████| 10/10 [02:03<00:00, 12.31s/it]

Mean Score: 0.7265674497392854
Std Score: 0.026565626386254284
Min Score: 0.6696035242290749
Max Score: 0.7766990291262136


In [101]:
model = ETC(1800, criterion="gini", max_depth=12, n_jobs=-1)
scores = model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True)


100%|██████████| 10/10 [02:11<00:00, 13.11s/it]

Mean Score: 0.7288466079831541
Std Score: 0.030193467747322445
Min Score: 0.6475770925110133
Max Score: 0.8106796116504854


In [ ]:
start, end = 0, 100
plt.figure(figsize=(20,5));
plt.bar(list(imps.keys())[start:end], list(imps.values())[start:end]);
plt.xticks(rotation=90);

# LightGBM

In [102]:
def lgbm_evaluator(model, X, y, file_lengths, ft_names, n_repeats=7, voting=True, verbose=True):
    scores = []
    num_range = trange(n_repeats) if verbose else range(n_repeats)
    for _ in num_range:
        for p1, p2, p3 in [(1,2,3), (2,3,1), (3,1,2)]:
            X_test, y_test = X[p1], y[p1]
            X_train = X[p2] + X[p3]
            y_train = y[p2] + y[p3]
            X_train, X_test = np.array(X_train), np.array(X_test)
            y_train, y_test = np.array(y_train), np.array(y_test)
            # train_data = lgb.Dataset(X_train, label=y_train, feature_name=ft_names)
            # test_data = lgb.Dataset(X_test, label=y_test, feature_name=ft_names)
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            if voting:
                filtered_pred = majority_voting(pred, file_lengths[p1])
                scores.append(accuracy_score(y_test, filtered_pred))
            else:
                scores.append(accuracy_score(y_test, pred))
    if verbose:
        print(f"Mean Score: {np.mean(scores)}")
        print(f"Std Score: {np.std(scores)}")
        print(f"Min Score: {np.min(scores)}")
        print(f"Max Score: {np.max(scores)}")
    return scores

In [13]:
get_streams, get_features, colnames = current_best()
ft_names = [i+j for i in colnames for j in ["_std", "_max", "_min", "_med"]]
overlap_rate, window_size = 0.75, 3000
X, y, file_lengths = get_processed_dataset(overlap_rate, window_size)

loading the data...	

100%|██████████| 151/151 [00:16<00:00,  9.00it/s]


extracting the features...  

100%|██████████| 736/736 [01:18<00:00,  9.43it/s]


In [104]:
model = lgb.LGBMClassifier(n_estimators=600, n_jobs=-1)
scores = lgbm_evaluator(model, X, y, file_lengths, ft_names, n_repeats=10, voting=True, verbose=True)

100%|██████████| 10/10 [05:20<00:00, 32.04s/it]

Mean Score: 0.6370220636964683
Std Score: 0.06150786171117007
Min Score: 0.5814977973568282
Max Score: 0.7227722772277227


In [107]:
for num_leaves in [45, 20]:
    print("num_leaves: ", num_leaves)
    model = lgb.LGBMClassifier(n_estimators=100, n_jobs=-1, objective="multiclass", num_leaves=num_leaves)
    scores = lgbm_evaluator(model, X, y, file_lengths, ft_names, n_repeats=10, voting=True, verbose=True)

num_leaves:  45


100%|██████████| 10/10 [03:54<00:00, 23.41s/it]


Mean Score: 0.6351950087442771
Std Score: 0.04569828538378031
Min Score: 0.5991189427312775
Max Score: 0.6996699669966997
num_leaves:  20


100%|██████████| 10/10 [03:52<00:00, 23.21s/it]

Mean Score: 0.5871531466905726
Std Score: 0.046134842274195266
Min Score: 0.5242718446601942
Max Score: 0.6336633663366337


# XGBoost

In [18]:
get_streams, get_features, colnames = current_best()
overlap_rate, window_size = 0.75, 3000
X, y, file_lengths = get_processed_dataset(overlap_rate, window_size)

loading the data...	

100%|████████████████████████████████████████████████████████████████████████████████| 151/151 [00:28<00:00,  5.21it/s]


extracting the features...  

100%|████████████████████████████████████████████████████████████████████████████████| 736/736 [00:43<00:00, 16.83it/s]


In [12]:
def xgb_evaluator(model, X, y, file_lengths, n_repeats=7, voting=True, verbose=True):
    scores = []
    num_range = trange(n_repeats) if verbose else range(n_repeats)
    for _ in num_range:
        for p1, p2, p3 in [(1,2,3), (2,3,1), (3,1,2)]:
            X_test, y_test = X[p1], y[p1]
            X_train = X[p2] + X[p3]
            y_train = y[p2] + y[p3]
            X_train, X_test = np.array(X_train), np.array(X_test)
            y_train, y_test = np.array(y_train), np.array(y_test)
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            if voting:
                filtered_pred = majority_voting(pred, file_lengths[p1])
                scores.append(accuracy_score(y_test, filtered_pred))
            else:
                scores.append(accuracy_score(y_test, pred))
    if verbose:
        print(f"Mean Score: {np.mean(scores)}")
        print(f"Std Score: {np.std(scores)}")
        print(f"Min Score: {np.min(scores)}")
        print(f"Max Score: {np.max(scores)}")
    return scores

In [26]:
def scaled_model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True):
    scores = []
    num_range = trange(n_repeats) if verbose else range(n_repeats)
    for _ in num_range:
        for p1, p2, p3 in [(1,2,3), (2,3,1), (3,1,2)]:
            X_test, y_test = X[p1], y[p1]
            X_train = X[p2] + X[p3]
            y_train = y[p2] + y[p3]
            scaler = MinMaxScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.fit_transform(X_test)
            # print(f"training model for person {p1}/3...", end="\t")
            model.fit(X_train_scaled, y_train)
            pred = model.predict(X_test_scaled)
            if voting:
                filtered_pred = majority_voting(pred, file_lengths[p1])
                scores.append(accuracy_score(y_test, filtered_pred))
            else:
                scores.append(accuracy_score(y_test, pred))
    if verbose:
        print(f"Mean Score: {np.mean(scores)}")
        print(f"Std Score: {np.std(scores)}")
        print(f"Min Score: {np.min(scores)}")
        print(f"Max Score: {np.max(scores)}")
    return scores

In [34]:
model = xgb.XGBClassifier(colsample_bytree = 1, max_depth = 5, subsample = 1, n_estimators=100, learning_rate = 0.01, objective = "multi:softmax", eval_metric = "merror")
scores = scaled_model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]c:\users\aponaloy shahin\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\aponaloy shahin\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting

c:\users\aponaloy shahin\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\aponaloy shahin\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
 60%|█████████

Mean Score: 0.4818193893295492
Std Score: 0.0043133688089684085
Min Score: 0.47572815533980584
Max Score: 0.48514851485148514
